In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

defining nn

In [4]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2,60),
            nn.Tanh(),
            nn.Linear(60,60),
            nn.Tanh(),
            nn.Linear(60,60),
            nn.Tanh(),
            nn.Linear(60, 60),
            nn.Tanh(),
            nn.Linear(60, 60),
            nn.Tanh(),
            nn.Linear(60, 60),
            nn.Tanh(),
            nn.Linear(60, 60),
            nn.Tanh(),
            nn.Linear(60, 60),
            nn.Tanh(),
            nn.Linear(60, 3), #outputs u, v, p
            )
        self.init_weights()
    def init_weights(self):
            for m in self.layers:
                if isinstance(m, nn.Linear):
                    nn.init.xavier_normal_(m.weight)  # Xavier initialization
                    nn.init.zeros_(m.bias)  # Zero bias #improvisation for vanishing/exploding gradients'''

    def forward(self, x):
            x = x.requires_grad_(True)  # Ensure gradients are tracked
            return self.layers(x)


  

loss functions

In [5]:
def pde_loss(model, x, y, rho=1.0, mu=0.01):
    xy=torch.cat([x,y],dim=1)
    xy.requires_grad_(True)
    output=model(xy)
    u, v, p= output[:,0:1], output[:,1:2], output[:,2:3]

    #first order derivatives
    u_x = torch.autograd.grad(u, xy, grad_outputs=torch.ones_like(u), create_graph=True)[0][:, 0:1]
    u_y = torch.autograd.grad(u, xy, grad_outputs=torch.ones_like(u), create_graph=True)[0][:, 1:2]
    v_x = torch.autograd.grad(v, xy, grad_outputs=torch.ones_like(v), create_graph=True)[0][:, 0:1]
    v_y = torch.autograd.grad(v, xy, grad_outputs=torch.ones_like(v), create_graph=True)[0][:, 1:2]
    p_x = torch.autograd.grad(p, xy, grad_outputs=torch.ones_like(p), create_graph=True)[0][:, 0:1]
    p_y = torch.autograd.grad(p, xy, grad_outputs=torch.ones_like(p), create_graph=True)[0][:, 1:2]

    #second order derivatives
    u_xx = torch.autograd.grad(u_x, xy, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:, 0:1]
    u_yy = torch.autograd.grad(u_y, xy, grad_outputs=torch.ones_like(u_y), create_graph=True)[0][:, 1:2]
    v_xx = torch.autograd.grad(v_x, xy, grad_outputs=torch.ones_like(v_x), create_graph=True)[0][:, 0:1]
    v_yy = torch.autograd.grad(v_y, xy, grad_outputs=torch.ones_like(v_y), create_graph=True)[0][:, 1:2]

    #pde residuals
    continuity= u_x + v_y
    momentum_x = -p_x + mu * (u_xx + u_yy) - rho * (u * u_x + v * u_y)  # x-momentum
    momentum_y = -p_y + mu * (v_xx + v_yy) - rho * (u * v_x + v * v_y)  # y-momentum

    #loss function(mse for residuals)
    loss_pde = (continuity**2).mean() + (momentum_x**2).mean() + (momentum_y**2).mean()
    return loss_pde

pinn=PINN()
optimizer = optim.Adam(pinn.parameters(), lr=0.001)

#training data
x_train=torch.rand(1000,1,requires_grad=True)*2-1
y_train=torch.rand(1000,1,requires_grad=True)*2-1


training loop

In [ ]:
for epoch in range(5000):
    optimizer.zero_grad()
    loss = pde_loss(pinn, x_train, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")
